In [ ]:
import pandas as pd
import numpy as np
import pickle

## Load physician ID to NPI mapping

In [ ]:
file_in = open(r'data/physician_id_to_npi.pkl', 'rb')

physician_id_to_npi = pickle.load(file_in)

file_in.close()

## Load Prescription Data

In [ ]:
in_file = open(r'data/prescriptions.pkl', 'rb')
prescriptions = pickle.load(in_file)
in_file.close()

## Shaping the data

In order to get the data into a form we can use for modeling, I will be grouping the payments by each provider, summing up the total amount of payment received from each manufacturer. This will give me a feature for each manufacturer and the value will be the total amount of payment that provider recieved during 2014 from that manufacturer.

I will also have a feature for each drug, I will use a one-hot encoding scheme.

Each row in this dataset will be a provider-drug pair where the target variable will either be the total number of claims filed or the total day supply that was prescribed in 2014.

I will be building a regression model and interpreting the coefficient derived from the manufacturer features. If no significant coeffecient is found then there is little to no influence of those payments on their prescription rates. I hypothesize that the drugs name will be the most significant variable in predicting the prescription rate but that certain manufacturer payments will exhibit some influence over these values.

Below is the code I ran to get the eventual data matrix and target values. Processing this was **very** memory intensive. At one point it took up over 430GB of RAM.

```python
payments_grouped = payments.groupby(['Physician_Profile_ID', 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name']).agg({'Total_Amount_of_Payment_USDollars': 'sum'})

payments_grouped = payments_grouped.unstack(
    'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name').reset_index().fillna(0)

payments_grouped.columns = payments_grouped.columns.droplevel()
# create a named column for the Physician_Profile_ID
payments_grouped['Physician_Profile_ID'] = payments_grouped[[0]].values

# Insert the NPI into payments
payments_grouped = payments_grouped.merge(physician_id_to_npi, on='Physician_Profile_ID', copy=False)

prescriptions = prescriptions[['NPI', 'SPECIALTY_DESCRIPTION', 'DRUG_NAME', 'TOTAL_DAY_SUPPLY']]
# merge the payments records with prescription data
prescriptions = prescriptions.merge(payments_grouped, on='NPI', copy=False)

lbl = LabelEncoder()

# encode the specialty and drug labels
prescriptions['specialty'] = lbl.fit_transform(prescriptions.SPECIALTY_DESCRIPTION)
specialty_features = lbl.classes_
prescriptions['drug'] = lbl.fit_transform(prescriptions.DRUG_NAME)
drug_features = lbl.classes_

Y = prescriptions['TOTAL_DAY_SUPPLY']

data_columns = []
data_columns.extend(prescriptions.columns[5:-3].tolist())
data_columns.extend(prescriptions.columns[-2:])

# only take the columns we care about
# NOTE: calling 'drop' took up a lot of memory and this took up less
prescriptions = prescriptions[data_columns]

oh = OneHotEncoder(categorical_features=[-2,-1])

features = np.concatenate([specialty_features, drug_features, prescriptions.columns[:-2]])

X = oh.fit_transform(prescriptions)

```

I wrote X, Y, and features to disk using pickle.